In [1]:
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.metrics import (f1_score, auc, roc_curve, confusion_matrix,
                             roc_auc_score, matthews_corrcoef)
import seaborn as sns
mpl.style.use("seaborn-deep")

In [6]:
os.path.exists("../results/CNN/binary_class.csv")


True

# Full Results from DA Detection Paper
## Overview
This notebook can be run to reproduce all figues and results from the BHK lab's DA detection paper. The raw output from the CNN and the Sinogram-based detection (SBD) algorithm should be saved locally for this notebook to run locally.

## Performance Metrics
The following metrics are computed for this study.
### Binary Classification
#### Mathews Correlation Coefficient (MCC)
$$ \text{MCC} = {\frac {{\mathit {TP}}\cdot {\mathit {TN}}-{\mathit {FP}}\cdot {\mathit {FN}}}{\sqrt {({\mathit {TP}}+{\mathit {FP}})({\mathit {TP}}+{\mathit {FN}})({\mathit {TN}}+{\mathit {FP}})({\mathit {TN}}+{\mathit {FN}})}}} $$

#### AUC
...


### Multiclass Classification
#### Generalized MCC
$$
{\text{MCC}}={\frac {\sum _{k}\sum _{l}\sum _{m}C_{kk}C_{lm}-C_{kl}C_{mk}}{{\sqrt {\sum _{k}(\sum _{l}C_{kl})(\sum _{k'|k'\neq k}\sum _{l'}C_{k'l'})}}{\sqrt {\sum _{k}(\sum _{l}C_{lk})(\sum _{k'|k'\neq k}\sum _{l'}C_{l'k'})}}}}
$$

In [2]:
""" Get Metrics """
# Compute metrics for different methods
metrics = pd.DataFrame(data={"SBD":np.nan, "CNN":np.nan, "3_class": np.nan, 
                             "2_class_annotation": np.nan, "3_class_annotation": np.nan},
                       index=["TPR", "FPR", "TNR", "FNR", "MCC", "Weighted AUC", " Weighted F1"])

def get_metrics(labels, predictions, scores=None) :
    if max(labels) - 2 < 1.0e-5 :
        # If the labels are (0, 2), convert to (0, 1)
        labels = labels / 2.
        predictions = predictions / 2.
    
    # Compute overall stats
    num_y = (labels == 1).sum()
    num_n = (labels == 0).sum()
    
    # Compute confusion matrix
    C = confusion_matrix(labels, predictions)
    tn, fp, fn, tp = C.ravel()
        
    tpr = tp / num_y
    fpr = fp / num_n
    tnr = tn / num_n
    fnr = fn / num_y
    
    # MCC
    mcc = matthews_corrcoef(labels, predictions)
    
    # AUC
    if type(scores) == type(None) :
        print("No scores provided")
        AUC = roc_auc_score(labels, predictions, average='weighted')
    else :
        AUC = roc_auc_score(labels, scores, average='weighted')
    
    # F1 Score
    f1 = f1_score(labels, predictions, average='weighted')
    
    return [tpr, fpr, tnr, fnr, mcc, AUC, f1]


metrics

,SBD,CNN,3_class,2_class_annotation,3_class_annotation
TPR,NaN,NaN,NaN,NaN,NaN
FPR,NaN,NaN,NaN,NaN,NaN
TNR,NaN,NaN,NaN,NaN,NaN
FNR,NaN,NaN,NaN,NaN,NaN
MCC,NaN,NaN,NaN,NaN,NaN
Weighted AUC,NaN,NaN,NaN,NaN,NaN
Weighted F1,NaN,NaN,NaN,NaN,NaN


In [3]:
""" Data Loading """
# Model Output
model_output_dir = "../results/"
sbd_results_path = os.path.join(model_output_dir, "SBD/binary_class.csv") # Sinogram-based DA detection 
cnn_results_path = os.path.join(model_output_dir, "CNN/binary_class.csv") # CNN-based DA detection

# Make into DataFrames
bhk_data = pd.read_csv(bhk_results_path, index_col="patient_id")
mat_data = pd.read_csv(cnn_results_path, index_col="patient_id")



# PyRadiomic Feature and GTV-DA distance analysis
# File locations
labels_path = os.path.join(model_output_dir, "radcure_DA_labels.csv")
double_labels_path = os.path.join(model_output_dir, "double_labels.csv")

# Dental artifact labels (DAL)
df_dal = pd.read_csv(labels_path, index_col="patient_id", 
                     usecols=["patient_id", "has_artifact", "a_slice"])
da_labels = df_dal

# Load results of annotator agreement
double_labels = pd.read_csv(double_labels_path, index_col="patient_id")



""" SBD and Thresholding-based Location detection"""
threshld_loc_preds = os.path.join(model_output_dir, "loc_preds.json")
sinogram_loc_preds = os.path.join(model_output_dir, "bhk_preds.json")
import json
with open(threshld_loc_preds) as json_file:
    thr_loc_data = json.load(json_file)
with open(sinogram_loc_preds) as json_file:
    sin_loc_data = json.load(json_file)
    


FileNotFoundError: [Errno 2] File /home/colin/Documents/BHKLab/data/DA_detection_paper/final_results/bhk_preds.csv does not exist: '/home/colin/Documents/BHKLab/data/DA_detection_paper/final_results/bhk_preds.csv'

In [ ]:
"""Data Cleaning"""
# Combine all results into one data frame with Radiomic Features, CNN, SBD, labels
# Combine the results from Mattea and Colin's methods into one DF

""" CNN and SBD DA classification outputs """
mat_data["bhk_preds"] = bhk_data["prediction"]
mat_data["CNN_preds"] = mat_data["CNN_preds"] * 2
# mat_data = mat_data.drop(columns=["p_index", "manual_artifact_location"])

full_data = mat_data
pred_data = mat_data.dropna()

# Rename some columns
pred_data = pred_data.rename(columns={"manual_artifact_status": "DA_mag",
                                      "CNN_preds": "CNN_pred", "CNN_probs0":"CNN_prob0",
                                      "CNN_probs1":"CNN_prob1", "bhk_preds":"bhk_pred"})
# Add column for binary label
pred_data["bit_label"] = (pred_data["DA_mag"] > 0).astype(int) * 2


pred_data


In [ ]:
""" Compute some statistics for human classification agreement """

print("Number of patients with 2 human labels: ", len(double_labels))

# Binarize the labels ((1 or 2) == 1 and 0 == 0)
binA = (double_labels["A_label"] != 0).values.astype(int)
binB = (double_labels["B_label"] != 0).values.astype(int)


# print(get_metrics(double_labels["A_binary"].values, double_labels["B_binary"].values))
""" Mathews Correlation for Binary agreement """
metrics.loc["MCC", "2_class_annotation"] = matthews_corrcoef(binA, binB)

""" Matthews correlation coefficient for 3 class agreement"""
metrics.loc["MCC", "3_class_annotation"] = matthews_corrcoef(double_labels["A_label"].values, 
                                                             double_labels["B_label"].values)

metrics

In [ ]:
""" Sinogram Based Detection Evaluation """
metrics["SBD"] = get_metrics(pred_data["bit_label"].values, pred_data["bhk_pred"].values)


# Compute some stats about this data
print("Number of Images tested on:", len(pred_data["bhk_pred"]))
print("Number of Strong: ", sum(pred_data["DA_mag"] == 2))
print("Number of Weak: ",   sum(pred_data["DA_mag"] == 1))
print("Number of no DA: ",  sum(pred_data["DA_mag"] == 0))


In [ ]:
""" Sinogram-Based DA Location Detection and Thresholding DA location detection """
thr_loc_pred = []   # Contains a location prediction for every DA+ patient
sin_loc_pred = []   # COntains a location prediction for every img predictied as DA+ by SBD
loc_pids = []
loc_labels = []

labels = da_labels["a_slice"]

for key in thr_loc_data :
    
    thr_preds = thr_loc_data[key]
    sin_preds = sin_loc_data[key]
    label = int(labels.loc[int(key)] )
    
    # Thresholding location prediction
    if len(thr_preds) > 1 :
        # Find pred which is closest to label
        deltas = np.abs(np.array(thr_preds) - label)
        best_pred = thr_preds[np.argmin(deltas)]
        thr_loc_pred.append(best_pred)
    else :
        thr_loc_pred.append(thr_preds[0])
        
    # Sinogram location prediction
    if len(sin_preds) > 1 :
        sin_deltas = np.abs(np.array(sin_preds)+20 - label)
        sin_best_pred = sin_preds[np.argmin(sin_deltas)]+20
        sin_loc_pred.append(best_pred)
    elif len(sin_preds) == 1 :
        sin_loc_pred.append(sin_preds[0]+20)
    else :
        sin_loc_pred.append(np.nan)
    

    loc_pids.append(key)
    loc_labels.append(label)

    
    
loc_df = pd.DataFrame({"patient_id": loc_pids, 
                       "label": loc_labels, 
                       "thr_pred": thr_loc_pred,
                       "sin_pred": sin_loc_pred}).set_index("patient_id")

loc_df["thr_delta"] = loc_df["thr_pred"] - loc_df["label"]
loc_df["sin_delta"] = loc_df["sin_pred"] - loc_df["label"]
        
loc_df

In [ ]:
# Look at results
n_thr = len(loc_df)
exact = len(loc_df[loc_df["thr_delta"] == 0])         / n_thr
pm_5  = len(loc_df[np.abs(loc_df["thr_delta"]) < 5])  / n_thr
pm_10 = len(loc_df[np.abs(loc_df["thr_delta"]) < 10]) / n_thr
pm_15 = len(loc_df[np.abs(loc_df["thr_delta"]) < 15]) / n_thr
print("THRESHOLDING BASED LOCATION")
print("Tested on ", len(loc_df["thr_pred"].dropna()), "images")
print(exact, pm_5, pm_10, pm_15)
print("Mean:", loc_df["thr_delta"].mean())
print("Std:", loc_df["thr_delta"].std())

# Look at sinogram-based results
n_sin = len(loc_df["sin_delta"].dropna())
exact = len(loc_df[loc_df["sin_delta"] == 0])         / n_sin
pm_5  = len(loc_df[np.abs(loc_df["sin_delta"]) < 5])  / n_sin
pm_10 = len(loc_df[np.abs(loc_df["sin_delta"]) < 10]) / n_sin
pm_15 = len(loc_df[np.abs(loc_df["sin_delta"]) < 15]) / n_sin
print("\nSINOGRAM BASED LOCATION")
print("Tested on ", len(loc_df["sin_pred"].dropna()), "images")
print(exact, pm_5, pm_10, pm_15)

In [ ]:
# """ Make graphs based on above results """


# # Create list of percentages for each distance
# labeller, n_double = [], len(double_labels)
# label_diff = double_labels["raw_diff"]
# sin_diff, thr_diff = [], []

# limit = 40

# for i in range(0, limit) :
#     if i == 0 :
#         labeller.append(len(label_diff[label_diff == i].abs()) / n_double)
#         sin_diff.append(len(loc_df[loc_df["sin_delta"].abs() == i]) / n_sin)
#         thr_diff.append(len(loc_df[loc_df["thr_delta"].abs() == i]) / n_thr)

#     else :
#         labeller.append(len(label_diff[label_diff < i].abs()) / n_double)
#         sin_diff.append(len(loc_df[loc_df["sin_delta"].abs() < i]) / n_sin)
#         thr_diff.append(len(loc_df[loc_df["thr_delta"].abs() < i]) / n_thr)
        
# # divide all data by number of samples to get percentages
# mpl.style.use("classic")
# fig = plt.figure()
# fig.set_facecolor('white')
# plt.plot(np.arange(0, limit), labeller, "--", label="Labeller Agreement")
# plt.plot(np.arange(0, limit), sin_diff, label="Sinogram-based detection")
# plt.plot(np.arange(0, limit), thr_diff, label="Threshold-based detection")
# plt.title("Comparison of DA slice detection methods with labeller agreement")
# plt.xlabel("Distance from prediction to label [slices]")
# plt.ylabel("Proportion of Images With Agreeing DA Location")
# plt.legend(loc="center right")
# plt.ylim([0, 1])
# plt.xlim([0, 25])
# plt.xticks(np.arange(0, 25))
# plt.show()

In [ ]:
""" Violin plot for location data """
thr = loc_df["thr_delta"].dropna().values
sin = loc_df["sin_delta"].dropna().values
agr = double_labels["raw_diff"].values

means = [thr.mean(), sin.mean(), agr.mean()]
error = [thr.std(), sin.std(), agr.std()]
# mpl.style.use("classic")
plt.rc('font', family='sans-serif', size=15)

fig, ax = plt.subplots(nrows=1, ncols=1)
fig.set_facecolor('white')

ax.violinplot([thr, sin, agr], [0,0.5,1], points=300, widths=0.3,
                      showmeans=False, showextrema=False, showmedians=True)
ax.set_xticks([0,0.5,1])
ax.set_xticklabels(["Threshold-based", "Sinogram-based", "Annotator agreement"])
ax.set_ylabel("Number of slices between prediction and label")
ax.set_title("Comparison of DA location detectors")
plt.rc('font', family='serif', size=15)
plt.show()





In [ ]:
""" Seaborn Violin plots """
# mpl.style.use("classic")
# mpl.style.use("classic")
plt.rc('font', family='sans-serif', size=15)

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=[6, 4])
fig.set_facecolor('white')
ax = sns.violinplot(data=[thr, sin, agr])
ax.set_xticklabels(["Threshold-based", "Sinogram-based", "Annotator\nagreement"])
ax.set_ylabel("Slices between prediction and label")
ax.set_xlabel("Detection Method")
plt.show()

In [ ]:
""" Convolutional Neural Network Detection """

# Remove images that were used in CNN training
trg_data = pd.read_csv("/home/colin/Downloads/CNN_Full_preds.csv", 
                       usecols=["patient_id", "isDuplicate"], 
                       index_col="patient_id")
test_data = pred_data[trg_data.loc[:, "isDuplicate"] == False]

# Print Some stats
print("Number of CNN test Images Used: ", len(test_data))
print("Number of Strong: ", sum(test_data["DA_mag"] == 2))
print("Number of Weak: ", sum(test_data["DA_mag"] == 1))
print("Number of no DA: ", sum(test_data["DA_mag"] == 0))

# Get performance metrics
metrics["CNN"] = get_metrics(test_data["bit_label"].values, 
                             test_data["CNN_pred"].values, 
                             scores=test_data["CNN_prob1"].values)

# mpl.style.use("classic")



# ROC curve
fpr, tpr, thresholds = roc_curve(test_data["bit_label"].values, 
                                 test_data["CNN_prob1"].values,
                                 pos_label=2)

fig = plt.figure()
fig.set_facecolor('white')
plt.rc('font', family='sans-serif', size=15)
plt.rc('xtick', labelsize='x-small')
plt.rc('ytick', labelsize='x-small')

plt.plot(fpr, tpr, label="AUC = "+str(metrics.loc["Weighted AUC", "CNN"])[:5])
plt.plot([0,0.5, 1], [0, 0.5, 1], label="Random")
plt.legend(loc="lower right")
plt.title("Binary CNN ROC Curve")
plt.xlabel("False positive rate")
plt.ylabel("True positive rate")
plt.show()

metrics

## Building a 3-Class Classifier
![alt text](3_class_schematic.jpg "Schematic for a 3-class DA classifier")

In [ ]:
""" 3-Class DA Classifier Results """
# Algorithm for 3 class
multi_pred = []
for pid in test_data.index.values :   # Use CNN test data
    sinogram = test_data.loc[pid, "bhk_pred"]
    cnn = test_data.loc[pid, "CNN_pred"]
    
    # Sinogram first
    if sinogram == 2 :
        if cnn == 2 : 
            multi_pred.append(2)
        else : # CNN = 0
            multi_pred.append(0)
    else : # Sinogram = 0
        if cnn == 2 :
            multi_pred.append(1)
        else : # CNN = 0
            multi_pred.append(0)
            

test_data.loc[:, "multi_pred"] = multi_pred

# Print stats
print("Number of test images: ", len(test_data))
print("Number of strong", sum(test_data["DA_mag"] == 2))
print("Number of weak", sum(test_data["DA_mag"] == 1))
print("Number of none", sum(test_data["DA_mag"] == 0), "\n")

print("MCC For 3-Class Classifier")
three_class_mcc = matthews_corrcoef(test_data["DA_mag"], test_data["multi_pred"])
print(three_class_mcc)

# Add 3 class MCC to metrics df
metrics.loc["MCC", "3_class"] = three_class_mcc



In [ ]:

import matplotlib as mpl
# mpl.style.use('fast')
# mpl.style.use("classic")


plt.close('all')
plt.rc('font', family='sans-serif', size=15)
plt.rc('xtick', labelsize='x-small')
plt.rc('ytick', labelsize='x-small')
fig, ax = plt.subplots(figsize=[6, 4], facecolor="white")

metrics.loc["MCC", :].plot.bar(color=["#59A14F", "#ECC539", "#AA6A99", "#519DBC", "#F08333"], 
                               ax=ax)
# metrics.loc["MCC", :].plot.bar()
plt.ylabel("Matthews Correlation Coeficient (MCC)", fontsize=15)
plt.xlabel("Classification Method", fontsize=15)
plt.xticks([0, 1, 2, 3, 4], 
           ["SBD", "CNN", "Hybrid", "Annotator\n(binary)", "Annotator\n(3-class)"])
plt.show()



In [ ]:
""" Create other contingency tables"""
# Labeller agreement
print("Labeller Agreement")
print(pd.crosstab(double_labels["A_label"], double_labels["B_label"]), "\n")

# Sinogram detection
print("Sinogram binary detection")
print(pd.crosstab(pred_data["bit_label"], pred_data["bhk_pred"]), "\n")

# CNN detection
print("CNN binary detection")
print(pd.crosstab(test_data["bit_label"], test_data["CNN_pred"] ), "\n")

# Hybrid 3-class detection
print("Hybrid 3-class detection")
print(pd.crosstab(test_data["DA_mag"], test_data["multi_pred"]) )